In [1]:
import pandas as pd
import requests
from config import api_key
import matplotlib.pyplot as plt

In [2]:
cityblocks_df = pd.read_csv('Resources/GeoLite2-City-Blocks-IPv6.csv')
locations_df = pd.read_csv('Resources/GeoLite2-City-Locations-en.csv')
# cityblocks_df= cityblocks_df.dropna()
# locations_df= locations_df.dropna()
# cityblocks_df

In [3]:
cityblocks_df = cityblocks_df.loc[:, ['geoname_id', 'latitude', 'longitude']]
cityblocks_df= cityblocks_df.dropna()
cityblocks_df

,geoname_id,latitude,longitude
0,2657896,47.3664,8.5546
1,5332921,37.2502,-119.7513
2,1848354,35.5598,139.6484
3,1861060,36.0000,138.0000
4,1861060,36.0000,138.0000
...,...,...,...
474010,953987,-29.0000,24.0000
474011,953987,-29.0000,24.0000
474012,953987,-29.0000,24.0000
474013,2328926,10.0000,8.0000


In [4]:
locations_df = locations_df.loc[:, ['city_name', 'geoname_id']]
locations_df= locations_df.dropna()
locations_df

,city_name,geoname_id
0,Protaras,18918
2,Oddur,49747
4,Mogadishu,53654
5,Merca,54225
6,Kismayo,55671
...,...,...
120604,Barrio Bicentenario,12112408
120605,Newry,12113105
120606,Nucleo Rural Tabatinga,12115717
120607,Pisek u Jablunkova,12122813


In [5]:
mergedDf = locations_df.merge(cityblocks_df,on='geoname_id')
mergedDf= mergedDf.dropna()
mergedDf = mergedDf.drop_duplicates()
mergedDf

,city_name,geoname_id,latitude,longitude
0,Mosul,99072,36.3350,43.1189
1,Ras Tanura,102891,26.6439,50.1592
2,Makkah,104515,21.4267,39.8261
6,Jeddah,105343,21.5168,39.2192
26,Ha'il,106281,27.5219,41.6907
...,...,...,...,...
204643,Jose C. Paz,12076997,-34.5029,-58.7512
204648,Bath,12089206,46.4758,-67.5712
204649,Balmoral,12096348,47.6790,-66.4342
204650,Tobique Narrows,12096889,46.7936,-67.6854


In [6]:
#Given the original dataset was over 47,000 and we are only looking for 500 cities which makes only %1.05 of the dataset. Took a random sample being confident that lats and longs will be randomized. 
merged500Df = mergedDf.sample(n = 500) 
merged500Df

,city_name,geoname_id,latitude,longitude
134231,La Paz,4000900,24.1244,-110.2964
72913,Neerpelt,2790357,51.2329,5.4421
43104,Richmond,2151650,-33.5980,150.7513
195908,Waimate,6214808,-44.8333,171.1167
97388,Chantepie,3026842,48.0889,-1.6152
...,...,...,...,...
107271,Vez,3062962,49.5673,15.4667
179947,Laguna Niguel,5364329,33.5157,-117.7109
27754,Mihara,1857334,34.3871,133.0684
9303,Puchaczow,760951,51.3104,22.9777


In [7]:
merged500Df["Temperature"] = ""
merged500Df["Humidity"] = ""
merged500Df["Cloudiness"] = ""
merged500Df["Wind Speed (mph)"] = ""

In [8]:
mergedDf

,city_name,geoname_id,latitude,longitude
0,Mosul,99072,36.3350,43.1189
1,Ras Tanura,102891,26.6439,50.1592
2,Makkah,104515,21.4267,39.8261
6,Jeddah,105343,21.5168,39.2192
26,Ha'il,106281,27.5219,41.6907
...,...,...,...,...
204643,Jose C. Paz,12076997,-34.5029,-58.7512
204648,Bath,12089206,46.4758,-67.5712
204649,Balmoral,12096348,47.6790,-66.4342
204650,Tobique Narrows,12096889,46.7936,-67.6854


In [13]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"
params = {"appid": api_key}

for index, row in merged500Df.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    
    params["lat"] = lat
    params["lon"] = lon
    
    response = requests.get(base_url,params).json()
        
    try:
        print(f"Grabbing the info.")
        merged500Df.loc[index,"Temperature"] = response['main']['temp']
        merged500Df.loc[index, "Humidity"] = response['main']['humidity']
        merged500Df.loc[index, "Cloudiness"] = response['clouds']['all']
        merged500Df.loc[index, "Wind Speed (mph)"] = response['wind']['speed']
        
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")

    print("------------")

Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing the info.
------------
Grabbing

In [14]:
merged500Df

,city_name,geoname_id,latitude,longitude,Temperature,Humidity,Cloudiness,Wind Speed (mph)
134231,La Paz,4000900,24.1244,-110.2964,292.59,67,0,2.24
72913,Neerpelt,2790357,51.2329,5.4421,281.07,61,72,2.1
43104,Richmond,2151650,-33.5980,150.7513,295,40,0,2.1
195908,Waimate,6214808,-44.8333,171.1167,288.71,93,56,1.34
97388,Chantepie,3026842,48.0889,-1.6152,285.1,93,47,1.5
...,...,...,...,...,...,...,...,...
107271,Vez,3062962,49.5673,15.4667,278.16,86,0,0.45
179947,Laguna Niguel,5364329,33.5157,-117.7109,287.49,85,44,2.73
27754,Mihara,1857334,34.3871,133.0684,287.21,76,20,7.7
9303,Puchaczow,760951,51.3104,22.9777,274.15,55,39,1
